## Load necessary libraries and paths

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys import ecephys_session
%matplotlib inline

# fix slow autocomplete
%config Completer.use_jedi = False

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # OS X 
    data_root = "/Volumes/Brain2019/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn1' in platstring):
    # then on AWS
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2019/"

manifest_path = os.path.join(data_root, "dynamic-brain-workshop/visual_coding_neuropixels/2019/manifest.json")
cache = EcephysProjectCache.fixed(manifest=manifest_path)
sessions = cache.get_sessions()

- ## Choose brain region (*areas*). This will produce *area_sessions*, the sessions where there are cells with this brain region
- ## Choose *session*. This chunk will locate the probe with the most cells and locate an LFP channel from it

In [ ]:
area = ['CA']
area_sessions = [] # Adds all the sessions that have the desired area (designated above)
for i in np.arange(len(sessions.structure_acronyms)):
    sessionid = sessions.structure_acronyms.index[i]
    if any(elem in sessions.structure_acronyms[sessionid] for elem in area):
        area_sessions.append(sessionid)

session = cache.get_session_data(area_sessions[0])
units = session.units
sampling_rate = 2500 # The sampling rate is very stable across probes

# Just CA units
chosenUnits = units[units.structure_acronym ==area[0]]
probe = chosenUnits.probe_id.value_counts().index[0] # Probe with most units from brain region
probeLFP = session.get_lfp(probe)
channelInfo = cache.get_channels()
channel = channelInfo[(channelInfo.ecephys_probe_id == probe) & (channelInfo.manual_structure_acronym == area[0])].index[0]
channelLFP = probeLFP.loc[dict(channel=channel)]
LFP = channelLFP
#LFP = channelLFP.loc[dict(time=slice(0,2000))]

## Plot raw LFP with underlying spectrogram underneath

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20,10))
ax[0].plot(LFP.values, c='steelblue', alpha=0.8)
ax[1].specgram(LFP,
             Fs=1 / (LFP.time.values[1] - LFP.time.values[0]),
             cmap='Spectral_r', aspect='auto')
plt.ylim(0,50)
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

## Plot periodogram for the LFP trace

In [ ]:
from scipy import signal
t = LFP.time.values
v = LFP.values
fs = 1/(t[1]-t[0])
f, Pxx_den = signal.welch(LFP, fs, nperseg=10**5)
plt.figure(figsize=(10,10))
plt.semilogy(f, Pxx_den)
plt.xlim(0,50)
plt.ylim(10**2,10**6)
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()